# State Pattern

The State Pattern is a behavioral design pattern that allows an object to alter its behavior when its internal state changes. The object will appear to change its class. This pattern is particularly useful when an object can be in one of several states, and its behavior varies depending on the current state.

## Intent:

**Encapsulation of State**: 
Encapsulate the behavior associated with a particular state into separate state objects and delegate to the current state object to handle state-specific behavior.
State Transitions: Allow an object to change its behavior when its internal state changes, effectively changing its class.

## Example: A Gumball Machine

Imagine a gumball machine that dispenses gumballs when a coin is inserted and the crank is turned. The machine has several states:

* No Coin: The machine is waiting for a coin.
* Has Coin: The machine has received a coin and is waiting for the crank to be turned.
* Sold: The machine is dispensing a gumball.
* Sold Out: The machine has no more gumballs.

In [1]:
from abc import ABC, abstractmethod

class State(ABC):
    @abstractmethod
    def insert_coin(self):
        pass

    @abstractmethod
    def eject_coin(self):
        pass

    @abstractmethod
    def turn_crank(self):
        pass

    @abstractmethod
    def dispense(self):
        pass

In [2]:
class NoCoinState(State):
    def __init__(self, gumball_machine):
        self.gumball_machine = gumball_machine

    def insert_coin(self):
        print("You inserted a coin.")
        self.gumball_machine.set_state(self.gumball_machine.get_has_coin_state())

    def eject_coin(self):
        print("You haven't inserted a coin.")

    def turn_crank(self):
        print("You turned, but there's no coin.")

    def dispense(self):
        print("You need to pay first.")

In [3]:
class HasCoinState(State):
    def __init__(self, gumball_machine):
        self.gumball_machine = gumball_machine

    def insert_coin(self):
        print("You can't insert another coin.")

    def eject_coin(self):
        print("Coin returned.")
        self.gumball_machine.set_state(self.gumball_machine.get_no_coin_state())

    def turn_crank(self):
        print("You turned...")
        self.gumball_machine.set_state(self.gumball_machine.get_sold_state())

    def dispense(self):
        print("No gumball dispensed.")

In [4]:
class SoldState(State):
    def __init__(self, gumball_machine):
        self.gumball_machine = gumball_machine

    def insert_coin(self):
        print("Please wait, we're already giving you a gumball.")

    def eject_coin(self):
        print("Sorry, you already turned the crank.")

    def turn_crank(self):
        print("Turning twice doesn't get you another gumball!")

    def dispense(self):
        self.gumball_machine.release_ball()
        if self.gumball_machine.get_count() > 0:
            self.gumball_machine.set_state(self.gumball_machine.get_no_coin_state())
        else:
            print("Oops, out of gumballs!")
            self.gumball_machine.set_state(self.gumball_machine.get_sold_out_state())

In [5]:
class SoldOutState(State):
    def __init__(self, gumball_machine):
        self.gumball_machine = gumball_machine

    def insert_coin(self):
        print("You can't insert a coin, the machine is sold out.")

    def eject_coin(self):
        print("You can't eject, you haven't inserted a coin yet.")

    def turn_crank(self):
        print("You turned, but there are no gumballs.")

    def dispense(self):
        print("No gumball dispensed.")

In [ ]:
class GumballMachine:
    def __init__(self, number_gumballs):
        self.sold_out_state = SoldOutState(self)
        self.no_coin_state = NoCoinState(self)
        self.has_coin_state = HasCoinState(self)
        self.sold_state = SoldState(self)

        self.state = self.sold_out_state if number_gumballs == 0 else self.no_coin_state
        self.count = number_gumballs

    def insert_coin(self):
        self.state.insert_coin()

    def eject_coin(self):
        self.state.eject_coin()

    def turn_crank(self):
        self.state.turn_crank()
        self.state.dispense()

    def set_state(self, state):
        self.state = state

    def release_ball(self):
        if self.count > 0:
            print("A gumball comes rolling out the slot...")
            self.count -= 1

    def get_count(self):
        return self.count

    def get_sold_out_state(self):
        return self.sold_out_state

    def get_no_coin_state(self):
        return self.no_coin_state

    def get_has_coin_state(self):
        return self.has_coin_state

    def get_sold_state(self):
        return self.sold_state

    def __str__(self):
        return f"\nMighty Gumball, Inc.\nInventory: {self.count} gumball{'s' if self.count != 1 else ''}\nMachine is {self.state.__class__.__name__}\n"
